МФТИ ФИВТ: Потапова Софья, 394 группа. 

In [1]:
from IPython.display import display, Math, Latex

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
- Hastie, The Elements of Statistical Learning, https://goo.gl/k3wfEU
    - 2.9 Model Selection and the Bias–Variance Tradeoff 
    - 15 Random Forests
- Соколов, Семинары по композиционным методам, https://goo.gl/sn8RyJ
- Andrew Ng, Bias vs. Variance, https://goo.gl/1ISZ6Y

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2016_fall <номер_группы> <фамилия>``, к примеру -- ``ML2016_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_401_task1.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2016_fall Question <Содержание вопроса>``

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было 

<h1 align="center">Check Questions</h1> 

Ответе на вопросы своими словами (загугленный материал надо пересказать), ответ обоснуйте (напишите и ОБЪЯСНИТЕ формулки если потребуется), если не выходит, то вернитесь к лекции дополнительным материалам:

**Вопрос 1**: Какие формулы у шума, смещения, разброса? Какой смысл у этих компонент?
$$Noise = E_{x,y}(a^*(x) - y)^2$$
$$Bias = E_{x,y}(a(x) - a^*(x))^2$$
$$Variance = E_{x,y}E_{X^l}(\mu(X^l(x))a(x))^2$$
Шум - ошибки в данных, неточности модели.
Смещение - показатель, при котором модель будет производить одинаковые ошибки для входа, независимо от обучающего множества. Модель смещает свои собственные предположения о реальных отношениях из-за связей, имеющихся в обучающих данных.
Разброс - мера отклонения от математического ожидания. Модель с большим разбросом, наоборот, будет производить различные ошибки для входа в зависимости от обучающего множества.

**Вопрос 2**: 4. Приведите пример семейства с маленьким смещением и большим разбросом. Приведите пример семейства с большим смещением и маленьким разбросом.

Маленькое смещение + большой разброс - деревья.
Болшое смещение и маленький разброс - линейные модели.

**Вопрос 3**: Как сгенерировать подвыборку с помощью бутстрапа?

Взять l элементов с повторениями (l <= размер выборки).

**Вопрос 4**: Что такое бэггинг?

Bootstrap Aggregating

Базовые алгоритмы b_t(x) обучаются независмо по случайным подвыборкам длины l с повторениями (как в методе bootstrap)

**Вопрос 5**:  Как соотносятся смещение разброс композиции, построенной с помощью бэггинга, со смещением и разбросом одного базового алгоритма?

Смещение одинаковое, разброс композиции с помощью бэггинга меньше, чем разброс одного базового алгоритма.


**Вопрос 6**: Как обучается случайный лес? В чем отличия от обычной процедуры построения решающих деревьев?

Беггинг над случайными деревьями, прунинг не производится, для обучения испльзуется подвыборка объектов, признак в каждой вершине дерева выбирается из случайного подмножества признаков (лекции Воронцова).

**Вопрос 7**: Почему хорошими базовыми алгоритмами для бэггинга являются именно деревья?

На деревьях большой разброс, но маленькое смещение. Поэтому при использовании беггинга становится маленьким и разброс, и смещение, и мы победили. 

**Вопрос 8**: Как оценить качество случайного леса с помощью out-of-bag-процедуры?

out-of-bag нужен для вычисления T - количества деревьев в лесу. Минимизируем число ошибок на объектах x_i, если не учитывать голоса деревьев, для которых x_i был обучающим. Качество случайного леса определяем количеством таких ошибок. 


-----------
PS: Если проверяющий не понял ответ на большинство вопросов, то будет пичалька. Пишите так, чтобы можно было разобраться. 

<h1 align="center">Bagging</h1> 

Известно, что бэггинг плохо работает, если в качестве базовых классификаторов взять knn. Попробуем понять причины на простом примере.

Пусть дана выборка $X^l$ из $l$ объектов с ответами из множества $Y = \{−1, +1\}$. Будем рассматривать классификатор одного ближайшего соседа в качестве базового алгоритма. Построим с помощью бэггинга композицию длины $N$:

$$a_N(x) = sign(\sum_{n=1}^{N} b_n(x))$$

Оцените вероятность того, что ответ композиции на произвольном объекте x будет
отличаться от ответа одного классификатора ближайшего соседа, обученного по всей
выборке. Покажите, что эта вероятность стремится к нулю при N → ∞.

-----
**<Решение>**
Обозначим за P вероятность, что элемент в выборке. 
$$P = \frac{\overline{C_{k-1}^l}}{\overline{C_k^l}}$$
$$P \in (0,1)$$
Пусть для фиксированного элемента i соседей лежат в том же классе, что и ближайший. Тогда $$(1-P)^i$$ вероятность того, что ближайшие i соседей не попали в выборку. Чтобы ответ композиции на этом элементе
отличался от ответа одного классификатора ближайшего соседа, обученного по всей выборке, достаточно, чтоб ближайшие i соседей не попадали в выборку хотя бы в половине случаев. То есть вероятность этого равна
$$((1-P)^i)^{N/2}$$, откуда видно, что при стремлении N к бесконечности, вероятность стремится к нулю. Именно поэтому нет смысла запускать беггинг на kNN

<h1 align="center">Bagging Implementation</h1> 

Реализуйте беггинг.

In [19]:
import warnings

import numpy as np
from copy import deepcopy
from sklearn.base import ClassifierMixin, BaseEstimator

warnings.simplefilter("ignore")


class BaggingClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, n_estimators, items_rate=0.8, features_rate=0.8):
        """
        Parameters
        ----------
        base_estimator: sklearn.Classifier
            Базовый алгоритм, который можно обучить (есть метод fit).
            Для обучение композиции нужно много таких, можэно получить с помощю copy.deepcopy

        n_estimators: int
            Число алгоритмов в композиции

        items_rate: float > 0
            Доля объектов из трейна, на которой будет обучаться каждый базовый алгоритм

        features_rate: float > 0
            Доля фичей, на которой будет обучаться и применяться каждый базовый алгоритм
        """
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.items_rate = items_rate
        self.features_rate = features_rate
        self.class_cnt = 0

    def fit(self, X, y):
        """
        Метод должен обучить композицию алгоритмов, используя X, y как обучающую выборку.
        Не забудте реализорвать функционал выбора случайных объектов и фичей.

        Parameters
        ----------
        X: 2d np.array
        y: 1d np.array
        """
        self.class_cnt = y.max()
        
        self.estimators = []

        self.features_idx = []

        items_cnt, features_cnt = X.shape
        features_for_estimator = int(features_cnt * self.features_rate)
        items_for_estimator = int(items_cnt * self.items_rate)
                
        for i in range(self.n_estimators):
            #print("hey")
            estimator = deepcopy(self.base_estimator)
            rand_items = np.random.choice(items_cnt, items_for_estimator, replace=True)
            rand_features = np.random.choice(features_cnt, features_for_estimator, replace=False)
            X_for_est = X[rand_items[:, None], rand_features]
            y_for_est = y[rand_items]
            estimator.fit(X_for_est, y_for_est)
            self.estimators.append(estimator)
            self.features_idx.append(rand_features)
        
        return self

    def predict(self, X):
        """

        Parameters
        ----------
        X: 2d np.array матрица объекты признаки на которых нужно сказать ответ

        Returns
        -------
        y_pred: 1d np.array, Вектор классов для каждого объекта
        """
        
        probs = [] # Храните тут ответы каждого базового алгоритма 

        counts = np.zeros((self.class_cnt + 1, X.shape[0]))
        for i in range(self.n_estimators):            
            probs = self.estimators[i].predict(X[:, self.features_idx[i]])
            for j in range(len(probs)):
                counts[probs[j]][j] += 1
        y_pred = np.argmax(counts, axis=0)
        return y_pred
    
        #return y_pred.astype("uint8")

### Titanic Test

In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

titanic = pd.read_csv('./data/train.csv')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

sex_encoder = LabelEncoder()
titanic.Sex = sex_encoder.fit_transform(titanic.Sex)
features = ['Pclass', 'Sex', 'Age', 'Fare']

In [15]:
X, y = titanic[features].values, titanic.Survived.values
X = np.nan_to_num(X)
X_train, y_train, X_test, y_test = X[:500], y[:500], X[500:], y[500:]

Нужно обучить свой беггинг на датасете титаник, и посмотреть работает ли он. 

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [8]:
# =======================================
# Обучите беггинг над DecisionTreeClassifier с 10 моделями
# =======================================

clf = BaggingClassifier(DecisionTreeClassifier(), 10)
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test))

0.928 0.808184143223


Проведите эксперименты:
    - Работает-ли беггинг лучше чем просто линейная модель?
    - Какой items_rate и features_rate работает лучше и почему?

In [36]:
# =======================================
# Обучите беггинг над DecisionTreeClassifier с 100 моделями
# =======================================

clf = BaggingClassifier(DecisionTreeClassifier(), 100)
clf.fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
print(acc)

(0.99550000000000005, 0.86099832815858612)


In [37]:
from sklearn.grid_search import GridSearchCV
grid_searcher = GridSearchCV(clf, scoring='accuracy', iid=False, cv=3, verbose=1, n_jobs=4,
param_grid={'items_rate':[0.1 * i for i in range(1,10)], 'features_rate':[0.25 * i for i in range(1,5)]})
grid_searcher.fit(X_train, y_train)
print('best params: %s,\nprediction accuracy: %s' % (
grid_searcher.best_params_, accuracy_score(grid_searcher.best_estimator_.predict(X_test), y_test)))

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   34.3s
[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:  2.6min finished


best params: {'items_rate': 0.7000000000000001, 'features_rate': 0.5},
prediction accuracy: 0.859326486745


In [38]:
clf = BaggingClassifier(DecisionTreeClassifier(), 10, 0.7, 0.5)
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test))

0.9487 0.852559509593


In [99]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
print(acc)

(0.80000000000000004, 0.76982097186700771)


Вывод: бэггинг работает лучше, чем линейная модель. items_rate = 0.7, features_rate = 0.5

### Adult Test

In [39]:
adult = pd.read_csv(
    './data/adult.data', 
    names=[
        "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
        "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
        "Hours per week", "Country", "Target"], 
    header=None, na_values="?")

adult = pd.get_dummies(adult)
adult["Target"] = adult["Target_ >50K"]
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values
X_train, y_train, X_test, y_test = X[:20000], y[:20000], X[20000:], y[20000:]

Ответте на вопросы:
    - Работает-ли беггинг лучше чем просто линейная модель?
    - Какой items_rate и features_rate работает лучше и почему?

In [40]:
clf = BaggingClassifier(DecisionTreeClassifier(), 100)
clf.fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
print(acc)

(0.99695, 0.86020221319958601)


In [41]:
grid_searcher = GridSearchCV(clf, scoring='accuracy', iid=False, cv=3, verbose=1, n_jobs=4,
param_grid={'items_rate':[0.1 * i for i in range(1,10)], 'features_rate':[0.25 * i for i in range(1,5)]})
grid_searcher.fit(X_train, y_train)
print('best params: %s,\nprediction accuracy: %s' % (
grid_searcher.best_params_, accuracy_score(grid_searcher.best_estimator_.predict(X_test), y_test)))

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   34.1s
[Parallel(n_jobs=4)]: Done 108 out of 108 | elapsed:  2.4min finished


best params: {'items_rate': 0.5, 'features_rate': 0.5},
prediction accuracy: 0.857575033835


In [42]:
clf = BaggingClassifier(DecisionTreeClassifier(), 100, 0.5, 0.5)
clf.fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
print(acc)

(0.94325000000000003, 0.85964493272828602)


In [113]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
print(acc)

(0.79990000000000006, 0.79515962104927951)


Вывод: беггинг работает лучше линейной модели. items_rate = feature_rate = 0.5

<h1 align="center">Text, Image Classification</h1> 

Дальше в каждом эксперименте нужно: 
- сравниться с линейной моделью ( какую лучше выбрать?=) )
- сделать выбор в пользу одной из моделей
- выбор обосновать, почему одна из моделей хуже а другая лучше
- что такое хуже и лучше
- попробуйте беггинг над деревьями и линейными моделями 
- почему работает или не работает, какие особенности данных на это влияют

### Text classification

In [145]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [146]:
vectorizer = TfidfVectorizer()
X_train, y_train = vectorizer.fit_transform(newsgroups_train.data), newsgroups_train.target
X_test,  y_test  = vectorizer.transform(newsgroups_test.data), newsgroups_test.target

In [5]:
%%time
# =======================================
# Обучите Линейную модель 
# =======================================
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test))

0.969860350009 0.827934147637
CPU times: user 17.7 s, sys: 1min 22s, total: 1min 40s
Wall time: 12.7 s


In [23]:
%%time
# =======================================
# Обучите беггинг над DecisionTreeClassifier
# =======================================

clf = BaggingClassifier(DecisionTreeClassifier(), 10, 0.5, 0.5)
clf.fit(X_train[:10000], y_train[:10000])
print(accuracy_score(clf.predict(X_train[:10000]), y_train[:10000]), accuracy_score(clf.predict(X_test), y_test))

0.925 0.465215082315
CPU times: user 53.3 s, sys: 1.73 s, total: 55 s
Wall time: 55 s


In [147]:
%%time
# =======================================
# Обучите беггинг над DecisionTreeClassifier
# =======================================

clf = BaggingClassifier(LogisticRegression(), 10, 0.5, 0.5)
clf.fit(X_train[:1000], y_train[:1000])
print(accuracy_score(clf.predict(X_train[:1000]), y_train[:1000]), accuracy_score(clf.predict(X_test), y_test))

0.823 0.376128518322
CPU times: user 56.3 s, sys: 32.7 s, total: 1min 28s
Wall time: 55.5 s


Здесь мы видим, что регрессия работает лучше, потому что количество фич достаточно велико, смещение мало, и фичи очень важны в анализе текстов(поэтому бэггинг плохо работает)

### Image classification

In [24]:
import os
import gzip
import numpy as np
from scipy import io
import _pickle as pickle

In [148]:
def load_cifar10(base='./data/cifar10'):
    def load_CIFAR_batch(filename):
        with open(filename, 'rb') as f:
            datadict = pickle.load(f, encoding='latin1')
            Y = np.array(datadict['labels'])
            X = datadict['data'].reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
            return X, Y

    def load_CIFAR10(ROOT):
        xs, ys = [], []
        for b in range(1, 6):
            f = os.path.join(ROOT, 'data_batch_%d' % (b,))
            X, Y = load_CIFAR_batch(f)
            xs.append(X)
            ys.append(Y)
        Xtr, Ytr = np.concatenate(xs), np.concatenate(ys)
        del X, Y
        Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
        return Xtr, Ytr, Xte, Yte

    cifar10_dir = os.path.join(base, 'cifar-10-batches-py')
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_test -= mean_image

    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2).copy()
    X_test = X_test.transpose(0, 3, 1, 2).copy()

    X_val, y_val = X_test, y_test

    return (X_train, y_train, X_val, y_val, X_test, y_test), X_train.shape[0], X_test.shape[0], (None, 3, 32, 32)

In [149]:
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10('./data/cifar10')[0]

In [150]:
X_train, X_test = X_train.reshape(X_train.shape[0], -1), X_test.reshape(X_test.shape[0], -1)

In [28]:
X_train[:10]

array([[ -71.71074,  -87.14036,  -81.05044, ...,   26.16942,  -29.90624,
         -42.38186],
       [  23.28926,   -4.14036,  -26.05044, ...,   25.16942,   28.09376,
          29.61814],
       [ 124.28926,  122.85964,  121.94956, ...,  -30.83058,  -30.90624,
         -30.38186],
       ..., 
       [-102.71074, -100.14036,  -98.05044, ...,  -13.83058,  -14.90624,
         -18.38186],
       [   3.28926,    0.85964,   -3.05044, ...,   22.16942,   23.09376,
          23.61814],
       [  -5.71074,  -20.14036,  -29.05044, ...,  -31.83058,  -29.90624,
         -28.38186]])

In [29]:
X_train.shape

(50000, 3072)

In [30]:
%%time
from sklearn.linear_model import LogisticRegression
# =======================================
# Обучите Линейную модель 
# =======================================
clf = LogisticRegression()
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test))

1.0 0.198
CPU times: user 56.7 s, sys: 1.76 ms, total: 56.7 s
Wall time: 56.7 s


In [31]:
%%time
# =======================================
# Обучите беггинг над DecisionTreeClassifier
# =======================================
clf = BaggingClassifier(DecisionTreeClassifier(), 100, 0.5, 0.5)
clf.fit(X_train[:2000], y_train[:2000])
print(accuracy_score(clf.predict(X_train[:2000]), y_train[:2000]), accuracy_score(clf.predict(X_test), y_test))

1.0 0.3373
CPU times: user 1min 36s, sys: 5.27 s, total: 1min 41s
Wall time: 1min 41s


Бэггинг выигрывает, несмотря на то, что используем не все фичи для обучения, что важно для картинок.
Но вообще результаты очень плохие, так как мы не учитываем соседние пиксели, а фичи между собой очень зависимы в распознавании изображений.

<h1 align="center">Random Forest Feature Impotance</h1> 

Опишите как вычисляется важность фичей в дереве, можите изучить как работает  feature\_importances_ в sklearn.

---

По выбранному критерию, например, Gini или энтропия. 
$$IGain(p,n) = h(\frac{P}{l}) - \frac{p+n}{l}h(\frac{p}{p+n}) - \frac{l - p - n}{l}h(\frac{P - p}{l - p - n})$$
$$h(q) = -q*log_2q - (1-q)*log_2(1-q)$$
$$IGini(p,n) = IGain(p,n) при h(q) = 4q(1-q)$$

Эти критерии считаются при разбиении на правую и левую, находясь в конкретном узле. Получется, фича тем важнее, чем она лучше разделяет выборку и чем она выше к корню.

Почитайте Feature Impotance для Adult и Titanic (используйте полный датасет), ПРОИНТЕРПРЕТИРУЙТЕ резульататы.

### Adult

In [64]:
X, y = adult[adult.columns[:-3]].values, adult[adult.columns[-1]].values
X_train, y_train, X_test, y_test = X[:20000], y[:20000], X[20000:], y[20000:]

In [65]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                        items_rate=1, features_rate=1).fit(X_train, y_train)
importancies = np.zeros(X_train.shape[1])
for i in range(100):
    importancies[clf.features_idx[i]] += clf.estimators[i].feature_importances_
importancies /= 100

In [108]:
df = pd.DataFrame(importancies)
df['features'] = np.array(adult.columns)[:-3]
df.sort(0, ascending=False)[:7]

,0,features
33,0.196999,Martial Status_ Married-civ-spouse
1,0.168104,fnlwgt
0,0.113710,Age
2,0.110225,Education-Num
3,0.104892,Capital Gain
5,0.064254,Hours per week
4,0.038680,Capital Loss


### titanic

In [134]:
titanic = pd.read_csv('./data/train.csv')[['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Parch']]
sex_encoder = LabelEncoder()
titanic.Sex = sex_encoder.fit_transform(titanic.Sex)
parch_encoder = LabelEncoder()
titanic.Parch = parch_encoder.fit_transform(titanic.Parch)
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Parch']

In [135]:
X, y = titanic[features].values, titanic.Survived.values
X = np.nan_to_num(X)
X_train, y_train, X_test, y_test = X[:500], y[:500], X[500:], y[500:]

In [137]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, 
                        items_rate=1, features_rate=1).fit(X_train, y_train)
importancies = np.zeros(X_train.shape[1])
for i in range(100):
    importancies[clf.features_idx[i]] += clf.estimators[i].feature_importances_
importancies /= 100

In [144]:
df = pd.DataFrame(importancies)
df['features'] = np.array(titanic.columns[1:])
df.sort(0, ascending=False)

,0,features
1,0.336504,Sex
3,0.308132,Fare
2,0.245023,Age
0,0.081032,Pclass
4,0.029308,Parch
